## Read the file 

In [1]:
import zipfile
import os
import pandas as pd

# Path to the zip file
zip_file_path = r"C:\Users\User\Desktop\Arcada\Introduction to MLDP\archive.zip"

# Directory to extract the files
extract_dir = r"C:\Users\User\Desktop\Arcada\Introduction to MLDP"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Assuming train.csv is in the root directory of the zip file, read it
csv_file_path = os.path.join(extract_dir, "train.csv")

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Display the first few rows of the dataframe
print(df.head())


                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


## Preprocessing

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Step 1: Read the CSV file into a Pandas DataFrame
#df = pd.read_csv("train.csv")

# Step 2: Preprocess the text data
df['Context'] = df['Context'].str.lower()
df['Response'] = df['Response'].str.lower()
df['Context'] = df['Context'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df['Response'] = df['Response'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# Replace missing values with a placeholder value
df.fillna('', inplace=True)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Context'] + ' ' + df['Response'])
df['Context'] = tokenizer.texts_to_sequences(df['Context'])
df['Response'] = tokenizer.texts_to_sequences(df['Response'])

# Step 4: Pad the sequences
max_seq_length = 50  # Choose an appropriate value
df['Context'] = pad_sequences(df['Context'], maxlen=max_seq_length, padding='post')
df['Response'] = pad_sequences(df['Response'], maxlen=max_seq_length, padding='post')

# Step 5: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['Context'], df['Response'], test_size=0.2, random_state=42)

# Now, X_train, X_val, y_train, and y_val contain the processed data ready for training the RNN model.


## Simple RNN and the Complex RNN

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

# Define the Simple RNN model
model = Sequential()
model.add(SimpleRNN(units=64, activation='tanh', input_shape=(max_seq_length, 1)))
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)




Epoch 1/10


88/88 [==============================] - 6s 32ms/step - loss: 8.0323 - accuracy: 0.0424 - val_loss: 6.7393 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 2s 22ms/step - loss: 6.1573 - accuracy: 0.0427 - val_loss: 6.5983 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 2s 21ms/step - loss: 5.9949 - accuracy: 0.0449 - val_loss: 6.5740 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 2s 22ms/step - loss: 5.9386 - accuracy: 0.0452 - val_loss: 6.5759 - val_accuracy: 0.0484
Epoch 5/10
88/88 [==============================] - 2s 22ms/step - loss: 5.9087 - accuracy: 0.0427 - val_loss: 6.5895 - val_accuracy: 0.0384
Epoch 6/10
88/88 [==============================] - 2s 22ms/step - loss: 5.8957 - accuracy: 0.0449 - val_loss: 6.6027 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 2s 22ms/step - loss: 5.8850 - accuracy: 0.0449 - val_loss: 6.6126 - val_accuracy: 0.0384
Epoch 8/1

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding, Dropout

# Define the SimpleRNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(SimpleRNN(units=128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(SimpleRNN(units=64, activation='tanh'))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)
# Print the model summary
print(model.summary())


Epoch 1/10
88/88 [==============================] - 8s 41ms/step - loss: 8.5293 - accuracy: 0.0370 - val_loss: 6.8193 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 3s 31ms/step - loss: 6.2207 - accuracy: 0.0441 - val_loss: 6.7505 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 3s 32ms/step - loss: 6.0259 - accuracy: 0.0413 - val_loss: 6.7135 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 3s 32ms/step - loss: 5.9380 - accuracy: 0.0441 - val_loss: 6.6931 - val_accuracy: 0.0484
Epoch 5/10
88/88 [==============================] - 3s 32ms/step - loss: 5.8996 - accuracy: 0.0441 - val_loss: 6.6792 - val_accuracy: 0.0484
Epoch 6/10
88/88 [==============================] - 3s 32ms/step - loss: 5.8770 - accuracy: 0.0402 - val_loss: 6.6908 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 3s 33ms/step - loss: 5.8562 - accuracy: 0.0431 - val_loss: 6.6879 - val_accuracy: 0.0384
Epoch 8/10
88

## Simple LSTM and the Complex LSTM

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_seq_length))
model.add(LSTM(units=64, activation='tanh', return_sequences=True))
model.add(LSTM(units=64, activation='tanh'))
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)




Epoch 1/10
88/88 [==============================] - 14s 51ms/step - loss: 9.5891 - accuracy: 0.0402 - val_loss: 7.9781 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 2s 27ms/step - loss: 6.4890 - accuracy: 0.0449 - val_loss: 6.7666 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 2s 28ms/step - loss: 5.9960 - accuracy: 0.0417 - val_loss: 6.6973 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 2s 27ms/step - loss: 5.9120 - accuracy: 0.0449 - val_loss: 6.6830 - val_accuracy: 0.0384
Epoch 5/10
88/88 [==============================] - 2s 27ms/step - loss: 5.8720 - accuracy: 0.0427 - val_loss: 6.6733 - val_accuracy: 0.0384
Epoch 6/10
88/88 [==============================] - 2s 27ms/step - loss: 5.8458 - accuracy: 0.0449 - val_loss: 6.6511 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 2s 27ms/step - loss: 5.8309 - accuracy: 0.0427 - val_loss: 6.6843 - val_accuracy: 0.0384
Epoch 8/10
8

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(LSTM(units=64, activation='tanh'))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)
# Print the model summary
print(model.summary())


Epoch 1/10
88/88 [==============================] - 14s 56ms/step - loss: 9.3880 - accuracy: 0.0417 - val_loss: 7.2852 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 3s 34ms/step - loss: 6.4069 - accuracy: 0.0438 - val_loss: 6.7632 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 3s 34ms/step - loss: 6.0540 - accuracy: 0.0449 - val_loss: 6.7382 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 3s 34ms/step - loss: 5.9608 - accuracy: 0.0434 - val_loss: 6.7369 - val_accuracy: 0.0384
Epoch 5/10
88/88 [==============================] - 3s 34ms/step - loss: 5.9138 - accuracy: 0.0449 - val_loss: 6.7126 - val_accuracy: 0.0384
Epoch 6/10
88/88 [==============================] - 3s 34ms/step - loss: 5.8827 - accuracy: 0.0420 - val_loss: 6.7082 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 3s 34ms/step - loss: 5.8577 - accuracy: 0.0449 - val_loss: 6.7038 - val_accuracy: 0.0384
Epoch 8/10
8

## Simple GRU and the Complex GRU

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense

# Define the GRU model
model = Sequential()
model.add(GRU(units=64, activation='tanh', input_shape=(max_seq_length, 1)))
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


Epoch 1/10
88/88 [==============================] - 7s 34ms/step - loss: 8.5089 - accuracy: 0.0413 - val_loss: 6.9237 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 2s 21ms/step - loss: 6.1654 - accuracy: 0.0427 - val_loss: 6.5649 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 2s 22ms/step - loss: 5.9071 - accuracy: 0.0434 - val_loss: 6.5385 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 2s 20ms/step - loss: 5.8239 - accuracy: 0.0424 - val_loss: 6.5343 - val_accuracy: 0.0384
Epoch 5/10
88/88 [==============================] - 2s 22ms/step - loss: 5.7746 - accuracy: 0.0449 - val_loss: 6.5369 - val_accuracy: 0.0384
Epoch 6/10
88/88 [==============================] - 2s 21ms/step - loss: 5.7408 - accuracy: 0.0409 - val_loss: 6.5398 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 2s 21ms/step - loss: 5.7131 - accuracy: 0.0449 - val_loss: 6.5495 - val_accuracy: 0.0384
Epoch 8/10
88

In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense, Embedding, Dropout

# Define the GRU model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(GRU(units=128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(GRU(units=64, activation='tanh'))
model.add(Dropout(0.2))  # Adding dropout regularization
model.add(Dense(units=len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to 3D
X_train_3d = np.expand_dims(X_train, axis=-1)
X_val_3d = np.expand_dims(X_val, axis=-1)

# Train the model for 10 epochs
history = model.fit(X_train_3d, np.array(y_train), validation_data=(X_val_3d, np.array(y_val)), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val_3d, np.array(y_val))
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

# Print the model summary
print(model.summary())


Epoch 1/10
88/88 [==============================] - 16s 59ms/step - loss: 8.8428 - accuracy: 0.0388 - val_loss: 6.8270 - val_accuracy: 0.0384
Epoch 2/10
88/88 [==============================] - 3s 36ms/step - loss: 6.2746 - accuracy: 0.0456 - val_loss: 6.7302 - val_accuracy: 0.0384
Epoch 3/10
88/88 [==============================] - 3s 37ms/step - loss: 6.0199 - accuracy: 0.0441 - val_loss: 6.6672 - val_accuracy: 0.0384
Epoch 4/10
88/88 [==============================] - 3s 37ms/step - loss: 5.9357 - accuracy: 0.0438 - val_loss: 6.6557 - val_accuracy: 0.0384
Epoch 5/10
88/88 [==============================] - 3s 37ms/step - loss: 5.8898 - accuracy: 0.0449 - val_loss: 6.6436 - val_accuracy: 0.0384
Epoch 6/10
88/88 [==============================] - 3s 37ms/step - loss: 5.8669 - accuracy: 0.0449 - val_loss: 6.6423 - val_accuracy: 0.0384
Epoch 7/10
88/88 [==============================] - 3s 38ms/step - loss: 5.8523 - accuracy: 0.0449 - val_loss: 6.6514 - val_accuracy: 0.0384
Epoch 8/10
8

## Evaluate 3 models with simple network and same number of nodes

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
df = pd.read_csv("train.csv")

# Drop rows with NaN values in the "Response" column
df = df.dropna(subset=['Response'])

# Convert the "Response" column to string type
df['Response'] = df['Response'].astype(str)

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Context'])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(df['Context'])
padded_input_sequences = pad_sequences(input_sequences)

# Label encode the "Response" column
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['Response'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_input_sequences, y_encoded, test_size=0.2, random_state=42)

# Determine the number of unique classes in the target variable
num_classes = len(np.unique(y_encoded))

# Build and train a basic RNN model
model_rnn = Sequential()
model_rnn.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(num_classes, activation='softmax'))
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Build and train an LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(num_classes, activation='softmax'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Build and train a GRU model
model_gru = Sequential()
model_gru.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_gru.add(GRU(100))
model_gru.add(Dense(num_classes, activation='softmax'))
model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

accuracy_rnn, precision_rnn, recall_rnn, f1_rnn = evaluate_model(model_rnn, X_test, y_test)
accuracy_lstm, precision_lstm, recall_lstm, f1_lstm = evaluate_model(model_lstm, X_test, y_test)
accuracy_gru, precision_gru, recall_gru, f1_gru = evaluate_model(model_gru, X_test, y_test)

print("Performance Metrics:")
print("Basic RNN - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_rnn, precision_rnn, recall_rnn, f1_rnn))
print("LSTM - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_lstm, precision_lstm, recall_lstm, f1_lstm))
print("GRU - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_gru, precision_gru, recall_gru, f1_gru))


Epoch 1/10
44/44 - 19s - loss: 7.8184 - accuracy: 7.1276e-04 - val_loss: 7.8736 - val_accuracy: 0.0000e+00 - 19s/epoch - 442ms/step
Epoch 2/10
44/44 - 16s - loss: 7.8028 - accuracy: 7.1276e-04 - val_loss: 7.7976 - val_accuracy: 0.0000e+00 - 16s/epoch - 364ms/step
Epoch 3/10
44/44 - 16s - loss: 7.8140 - accuracy: 0.0014 - val_loss: 7.8841 - val_accuracy: 0.0000e+00 - 16s/epoch - 364ms/step
Epoch 4/10
44/44 - 16s - loss: 7.8298 - accuracy: 3.5638e-04 - val_loss: 7.9706 - val_accuracy: 0.0000e+00 - 16s/epoch - 362ms/step
Epoch 5/10
44/44 - 16s - loss: 7.8360 - accuracy: 0.0000e+00 - val_loss: 7.8861 - val_accuracy: 0.0000e+00 - 16s/epoch - 365ms/step
Epoch 6/10
44/44 - 16s - loss: 7.8185 - accuracy: 0.0014 - val_loss: 8.0066 - val_accuracy: 0.0000e+00 - 16s/epoch - 364ms/step
Epoch 7/10
44/44 - 16s - loss: 7.8119 - accuracy: 0.0011 - val_loss: 8.1195 - val_accuracy: 0.0000e+00 - 16s/epoch - 359ms/step
Epoch 8/10
44/44 - 16s - loss: 7.7660 - accuracy: 0.0018 - val_loss: 7.9654 - val_accura

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


22/22 [==============================] - 5s 186ms/step


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


22/22 [==============================] - 5s 182ms/step
Performance Metrics:
Basic RNN - Accuracy: 0.0000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
LSTM - Accuracy: 0.0014, Precision: 0.0007, Recall: 0.0014, F1 Score: 0.0009
GRU - Accuracy: 0.0342, Precision: 0.0269, Recall: 0.0342, F1 Score: 0.0291


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Evaluate 3 models with complex network and different number of nodes

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
df = pd.read_csv("train.csv")

# Step 2: Preprocess the text data
df['Context'] = df['Context'].str.lower()
df['Response'] = df['Response'].str.lower()
df['Context'] = df['Context'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df['Response'] = df['Response'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# Replace missing values with a placeholder value
df.fillna('', inplace=True)

# Drop rows with NaN values in the "Response" column
df = df.dropna(subset=['Response'])

# Convert the "Response" column to string type
df['Response'] = df['Response'].astype(str)

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Context'])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(df['Context'])
padded_input_sequences = pad_sequences(input_sequences)

# Label encode the "Response" column
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['Response'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_input_sequences, y_encoded, test_size=0.2, random_state=42)

# Determine the number of unique classes in the target variable
num_classes = len(np.unique(y_encoded))

# Build and train a basic RNN model
model_rnn = Sequential()
model_rnn.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_rnn.add(SimpleRNN(128, return_sequences=True))  # Adding return_sequences=True for stacking RNN layers
model_rnn.add(SimpleRNN(128))  # Adding another RNN layer
model_rnn.add(Dense(64, activation='relu'))  # Adding a dense layer for more complexity
model_rnn.add(Dense(num_classes, activation='softmax'))
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Build and train an LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_lstm.add(LSTM(128, return_sequences=True))  # Adding return_sequences=True for stacking LSTM layers
model_lstm.add(LSTM(128))  # Adding another LSTM layer
model_lstm.add(Dense(64, activation='relu'))  # Adding a dense layer for more complexity
model_lstm.add(Dense(num_classes, activation='softmax'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Build and train a GRU model
model_gru = Sequential()
model_gru.add(Embedding(total_words, 100, input_length=X_train.shape[1]))
model_gru.add(GRU(256, return_sequences=True))  # Adding return_sequences=True for stacking GRU layers
model_gru.add(GRU(256))  # Adding another GRU layer
model_gru.add(Dense(128, activation='relu'))  # Adding a dense layer for more complexity
model_gru.add(Dense(num_classes, activation='softmax'))
model_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

accuracy_rnn, precision_rnn, recall_rnn, f1_rnn = evaluate_model(model_rnn, X_test, y_test)
accuracy_lstm, precision_lstm, recall_lstm, f1_lstm = evaluate_model(model_lstm, X_test, y_test)
accuracy_gru, precision_gru, recall_gru, f1_gru = evaluate_model(model_gru, X_test, y_test)

print("Performance Metrics:")
print("Basic RNN - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_rnn, precision_rnn, recall_rnn, f1_rnn))
print("LSTM - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_lstm, precision_lstm, recall_lstm, f1_lstm))
print("GRU - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_gru, precision_gru, recall_gru, f1_gru))


Epoch 1/10
44/44 - 27s - loss: 7.8447 - accuracy: 3.5600e-04 - val_loss: 7.8463 - val_accuracy: 0.0000e+00 - 27s/epoch - 611ms/step
Epoch 2/10
44/44 - 24s - loss: 7.8057 - accuracy: 3.5600e-04 - val_loss: 8.0537 - val_accuracy: 0.0000e+00 - 24s/epoch - 540ms/step
Epoch 3/10
44/44 - 23s - loss: 7.7612 - accuracy: 7.1200e-04 - val_loss: 8.1766 - val_accuracy: 0.0000e+00 - 23s/epoch - 523ms/step
Epoch 4/10
44/44 - 23s - loss: 7.7464 - accuracy: 3.5600e-04 - val_loss: 8.3952 - val_accuracy: 0.0000e+00 - 23s/epoch - 520ms/step
Epoch 5/10
44/44 - 23s - loss: 7.7272 - accuracy: 7.1200e-04 - val_loss: 8.4402 - val_accuracy: 0.0000e+00 - 23s/epoch - 513ms/step
Epoch 6/10
44/44 - 24s - loss: 7.7069 - accuracy: 0.0011 - val_loss: 8.6066 - val_accuracy: 0.0000e+00 - 24s/epoch - 534ms/step
Epoch 7/10
44/44 - 24s - loss: 7.6942 - accuracy: 0.0014 - val_loss: 8.6792 - val_accuracy: 0.0000e+00 - 24s/epoch - 536ms/step
Epoch 8/10
44/44 - 19s - loss: 7.6790 - accuracy: 0.0014 - val_loss: 9.0447 - val_ac

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


22/22 [==============================] - 5s 189ms/step


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


22/22 [==============================] - 9s 369ms/step
Performance Metrics:
Basic RNN - Accuracy: 0.0000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
LSTM - Accuracy: 0.0000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
GRU - Accuracy: 0.1878, Precision: 0.1636, Recall: 0.1878, F1 Score: 0.1707


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
print(model_rnn.summary())
print(model_lstm.summary())
print(model_gru.summary())


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 529, 100)          378800    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 529, 128)          29312     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 2480)              161200    
                                                                 
Total params: 610464 (2.33 MB)
Trainable params: 610464 (2.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "seque